In [53]:
import os
import json
import collections

In [2]:
os.listdir('../github_structured_data/')

['characters.csv', 'graph.json', 'relations.csv']

In [3]:
wands = []
with open('../wand/wands.json') as f:
    wands = json.load(f)

In [4]:
wands_revised = {}
for wand in wands:
    owner = wand['wand_owner']
    wand_info = {}
    wand_info['wood'] = wand['wand_wood']
    wand_info['length'] = wand['wand_length']
    wand_info['core'] = wand['wand_core']
    wand_info['flexibility'] = wand['wand_flexibility']
    
    wands_revised[owner] = wand_info

In [6]:
wands_key_lst = list(wands_revised.keys())

In [62]:
person_att_by_house = {}
for (dirpath, dirnames, filenames) in os.walk('../fandom/fandom_data'):
    for file in filenames:
        house_name = file[:-5]
        
        path = '../fandom/fandom_data/' + file
        with open(path) as f:
            value = json.load(f)
            
            flat_value = {}
            for val in value:
                name = val['name']
                flat_value[name] = val
            person_att_by_house[house_name] = flat_value

In [63]:
kaggle_data = []
with open('../kaggle_structured_data/characters.csv') as f:
    first_line = []
    first = True
    for line in f:
        lines = line[:-1].split(';')
        #print(lines)
        if first == True:
            first_line = lines
            first = False
            continue
        
        row = {}
        for i in range(len(lines)):
            row[first_line[i]] = lines[i]
        kaggle_data.append(row)

In [79]:
with open('../wikidata/wizards/data/info.json') as f:
    wizards_info = {}
    for i in json.load(f):
        wizards_info[i['characterLabel']] = i
with open('../wikidata/muggles/data/info.json') as f:
    muggles_info = {}
    for i in json.load(f):
        muggles_info[i['characterLabel']] = i
with open('../wikidata/others/data/info.json') as f:
    others_info = {}
    for i in json.load(f):
        others_info[i['characterLabel']] = i

In [81]:
wikidata_data_info = {}
wikidata_data_info.update(wizards_info)
wikidata_data_info.update(muggles_info)
wikidata_data_info.update(others_info)

In [85]:
with open('../wikidata/wizards/data/relation.json') as f:
    wizards_relation = {}
    for i in json.load(f):
        wizards_relation[i['characterLabel']] = i
with open('../wikidata/muggles/data/relation.json') as f:
    muggles_relation = {}
    for i in json.load(f):
        muggles_relation[i['characterLabel']] = i
with open('../wikidata/others/data/relation.json') as f:
    others_relation = {}
    for i in json.load(f):
        others_relation[i['characterLabel']] = i

In [86]:
wikidata_data_relation = {}
wikidata_data_relation.update(wizards_relation)
wikidata_data_relation.update(muggles_relation)
wikidata_data_relation.update(others_relation)

In [87]:
Gryffindors = {}
with open('../fandom/fandom_data/Gryffindors.json') as f:
    for i in json.load(f):
        Gryffindors[i['name']] = i
Hufflepuffs = {}
with open('../fandom/fandom_data/Hufflepuffs.json') as f:
    for i in json.load(f):
        Hufflepuffs[i['name']] = i
Ravenclaws = {}
with open('../fandom/fandom_data/Ravenclaws.json') as f:
    for i in json.load(f):
        Ravenclaws[i['name']] = i
Slytherins = {}
with open('../fandom/fandom_data/Slytherins.json') as f:
    for i in json.load(f):
        Slytherins[i['name']] = i

In [89]:
combined_houses = {}
combined_houses.update(Gryffindors)
combined_houses.update(Hufflepuffs)
combined_houses.update(Ravenclaws)
combined_houses.update(Slytherins)

# Info

In [61]:
def get_att(val, key):
    try:
        return val[key]
    except KeyError:
        return 'DNE'

In [148]:
from math import floor, ceil
def jaro_distance(s1, s2):
      
    # If the s are equal
    if (s1 == s2):
        return 1.0
  
    # Length of two s
    len1 = len(s1)
    len2 = len(s2)b
  
    # Maximum distance upto which matching
    # is allowed
    max_dist = floor(max(len1, len2) / 2) - 1
  
    # Count of matches
    match = 0
  
    # Hash for matches
    hash_s1 = [0] * len(s1)
    hash_s2 = [0] * len(s2)
  
    # Traverse through the first
    for i in range(len1):
  
        # Check if there is any matches
        for j in range(max(0, i - max_dist), 
                       min(len2, i + max_dist + 1)):
              
            # If there is a match
            if (s1[i] == s2[j] and hash_s2[j] == 0):
                hash_s1[i] = 1
                hash_s2[j] = 1
                match += 1
                break
  
    # If there is no match
    if (match == 0):
        return 0.0
  
    # Number of transpositions
    t = 0
    point = 0
  
    # Count number of occurances
    # where two characters match but
    # there is a third matched character
    # in between the indices
    for i in range(len1):
        if (hash_s1[i]):
  
            # Find the next matched character
            # in second
            while (hash_s2[point] == 0):
                point += 1
  
            if (s1[i] != s2[point]):
                point += 1
                t += 1
    t = t//2
  
    # Return the Jaro Similarity
    return (match/ len1 + match / len2 + 
            (match - t) / match)/ 3.0


In [156]:
def name_matching(n1, n2):
    n1_lst = n1.split(' ')
    n2_lst = n2.split(' ')
    
    if len(n1_lst) < len(n2_lst):
        for nn1 in n1_lst:
            if nn1 not in n2_lst:
                break
        else:
            return 1.0
    elif len(n1_lst) == len(n2_lst):
        for idx in range(len(n1_lst)):
            if n1_lst[idx] != n2_lst[idx]:
                break
        else:
            return 1.0
    else:
        for nn1 in n2_lst:
            if nn1 not in n1_lst:
                break
        else:
            return 1.0

    return jaro_distance(n1,n2)

In [158]:
matched_pairs = {}

for k1 in wikidata_data_info.keys():
    for k2 in combined_houses.keys():
        if name_matching(k1,k2) > 0.9:
            wikidata_val = wikidata_data[k1]
            fandom_val = combined_houses[k2]
            
            

Ron Weasley
Rose Granger-Weasley
1
----
Hermione Granger
Hermione Jean Granger
2
----
Severus Snape
Severus Snape
3
----
Draco Malfoy
Draco Lucius Malfoy
4
----
Ginny Weasley
Alannis Shepley
5
----
Neville Longbottom
Neville Longbottom
6
----
Minerva McGonagall
Minerva McGonagall
7
----
Albus Dumbledore
Albus Percival Wulfric Brian Dumbledore
8
----
Sirius Black
Sirius Black III
9
----
James Potter
James Sirius Potter
10
----
Lily Potter
Lily Luna Potter
11
----
Sybill Trelawney
Sullivan Fawley
12
----
Cho Chang
Cho Chang
13
----
Cedric Diggory
Cedric Diggory
14
----
Lucius Malfoy
Lucius Malfoy
15
----
Salazar Slytherin
Salazar Slytherin
16
----
Peter Pettigrew
Peter Pettigrew
17
----
Bellatrix Lestrange
Leta Lestrange
18
----
Garrick Ollivander
Garrick Ollivander
19
----
Gilderoy Lockhart
Gilderoy Lockhart
20
----
Narcissa Malfoy
Narcissa Malfoy (née Black)
21
----
Pomona Sprout
Pomona Sprout
22
----
Filius Flitwick
Filius Flitwick
23
----
Quirinus Quirrell
Quirinus Quirrell
24
----
P

In [161]:
wikidata_data_info['Perseus Parkinson']

{'character': 'http://www.wikidata.org/entity/Q65404692',
 'characterLabel': 'Perseus Parkinson',
 'genderLabel': 'male',
 'blood_statusLabel': 'pureblood',
 'employerLabel': 'Ministry of Magic'}

In [162]:
combined_houses['Pansy Parkinson']

{'name': 'Pansy Parkinson',
 'Biographical information': {'Born': 'Between 1 September 1979 and 31 August 1980',
  'Blood status': 'Pure-blood or Half-blood',
  'Nationality': 'British or Irish',
  'Title(s)': 'Prefect'},
 'Physical information': {'Species': 'Human', 'Gender': 'Female'},
 'Relationship information': {'Family members': 'Perseus Parkinson (possible relative)',
  'Romances': 'Draco Malfoy (ex-boyfriend)'},
 'Magical characteristics': {'Wand': 'Unknown length, wood and core'},
 'Affiliation': {'House': 'Slytherin',
  'Loyalty': ['Parkinson family',
   "Hogwarts School of Witchcraft and WizardrySlytherinInquisitorial SquadDraco Malfoy's gang",
   'Slytherin',
   'Inquisitorial Squad',
   "Draco Malfoy's gang",
   'Death Eaters (unofficial)']}}

#### Gryffindors

# Relation